# DSC 80: Project 02

### Checkpoint Due Date: Thursday, October 24, 11:59:59 PM (Q1-5)

### Final Due Date: Thursday, October 31 11:59:59 PM

---
# Instructions

This Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems.  
* Like the lab, your coding work will be developed in the accompanying `projectXX.py` file, that will be imported into the current notebook. This code will be autograded.
* The project also has free response questions. To answer the free response questions, edit the markdown cell where specified (as in DSC 10). Submission of the project include uploading a pdf of this notebook to gradescope for manual grading.

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the HW! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `projectXX.py` (much like we do in the notebook).
- Always document your code!

## Checkpoint Instructions

* The checkpoint requires you to turn in **questions 1-5**; 
* The checkpoint will be graded for *approximate* correctness: easier than the final tests; harder than the doctests.


In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import project02 as proj

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

# An Investigation into Flight Delays

### The flights dataset

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). There are data for the years 1987 - 2018. See the description of columns in `data/columns.txt`.

This project will look at a single year (2015) to keep the analysis "simple", which is available at the URL below (*NOT* on the data.gov site).


* To download the flights dataset to your computer, use [this link](https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip), unzip the file, and place `flights.csv` in your project directory.

* To download the dataset on `datahub.ucsd.edu` (this works on your computer as well!):
    - Open the terminal in datahub ("new > Terminal")
    - Change the directory to where you want your data (e.g. `cd [ASSIGNMENT_PATH]/data`)
    - Download the unzipped dataset using these commands:
        1. `wget https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip`
        2. `unzip flight-delays.zip`
    - `flights.csv` should be in the directory.
    
**NOTE: The unzipped files must be in the `project02/data` directory in order for the doctests to work!**

### Creating your datasets

**Question 1**

The flights dataset for 2015 is not small (~600MB). While you could likely load the entire dataset into Pandas on your laptop, if you wanted to work with more than one year, this would quickly become difficult (the data is available for 1987-2018). Therefore, we will filter down the dataset into two smaller files without ever reading the larger dataset fully into memory. We are going to create two smaller datasets:

1. All flights arriving or departing from San Diego International Airport in 2015.
2. All flights flown by either JetBlue or Southwest Airline in 2015.

---

To do this, you are going to use the `chunksize=N` keyword in Pandas `read_csv` to read the flights dataset in blocks of `N` lines. When you use this keyword argument, `pd.read_csv(fp, chunksize=N)` becomes a *iterator* that iterates through dataframes of length N until you have reached the end of the dataset. A typical pattern looks like:
```
L = pd.read_csv(filepath, chunksize=1000)
for df in L:
    process(df)
```
Where each `df` is a dataframe of length 1000. 

The processing you are going to do is:
1. Iterate through the dataset, chunk-by-chunk,
2. Filtering out rows of each chunk
3. Incrementally add to a filtered csv file (since the data is perhaps too big to keep in memory). Keep in mind, if you want to keep writing to the same file, the mode='a' keyword in the `.to_csv` method can be helpful when calling it in the loop (a stands for 'append')

---

Write two functions that create the datasets below, using the 'chunking' pattern described above. Your functions should use `chunksize` of 10000.
1. `get_san` which takes in a filepath containing all flights and an filepath where filtered dataset #1 is written. The function should return `None`.
1. `get_jb_sw` which takes in a filepath containing all flights and an filepath where filtered dataset #2 is written. The function should return `None`.

*Remark 1*: **Gradescope autograding servers are quite small and can't load this dataset into memory** -- so your code that reads in the large `flights.csv` dataset *must* work with chunks of the dataset one at a time to pass!

*Remark 2*: You can check your work using the datasets included in the zip file!

In [17]:
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'santest.tmp')
proj.get_san(infp, outfp)
df = pd.read_csv(outfp)
print(df.shape)
os.remove(outfp)

(53, 31)


# Flight Delays to/from San Diego

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). 

The zip file at the [URL](https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip) contains a file `to_from_san.csv` that consists of all flights either to or from SAN (San Diego) in 2015 -- i.e. the output of Question 1. This dataset should match the dataset that your code returned in question 1.

Read in `to_from_san.csv` using `read_csv` and inspect the dataframe for an initial assessment about the data quality.

In [18]:
# Run this cell
to_from_san_filepath = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(to_from_san_filepath)
flights.head(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,1,4,UA,1568,N24212,SAN,EWR,620,...,1358.0,-42.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,1,1,4,AA,1390,N496AA,SAN,DFW,620,...,1101.0,-24.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,1,4,NK,470,N534NK,PDX,SAN,620,...,828.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,1,4,DL,1592,N723TW,SAN,ATL,629,...,1311.0,-19.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,1,4,US,567,N524UW,SAN,PHX,630,...,900.0,7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140414 entries, 0 to 140413
Data columns (total 31 columns):
YEAR                   140414 non-null int64
MONTH                  140414 non-null int64
DAY                    140414 non-null int64
DAY_OF_WEEK            140414 non-null int64
AIRLINE                140414 non-null object
FLIGHT_NUMBER          140414 non-null int64
TAIL_NUMBER            140205 non-null object
ORIGIN_AIRPORT         140414 non-null object
DESTINATION_AIRPORT    140414 non-null object
SCHEDULED_DEPARTURE    140414 non-null int64
DEPARTURE_TIME         139010 non-null float64
DEPARTURE_DELAY        139010 non-null float64
TAXI_OUT               138959 non-null float64
WHEELS_OFF             138959 non-null float64
SCHEDULED_TIME         140414 non-null int64
ELAPSED_TIME           138564 non-null float64
AIR_TIME               138564 non-null float64
DISTANCE               140414 non-null int64
WHEELS_ON              138864 non-null float64
TAXI_IN         

In [20]:
flights.describe()

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,140414.0,140414.000000,140414.000000,140414.000000,140414.000000,140414.000000,139010.000000,139010.000000,138959.000000,138959.000000,...,140414.000000,138864.000000,138564.000000,140414.000000,140414.000000,25257.000000,25257.000000,25257.000000,25257.000000,25257.000000
mean,2015.0,6.213120,15.714622,3.931510,1748.363497,1342.582385,1352.533739,8.893698,14.879691,1376.431386,...,1526.666137,1514.229462,3.502937,0.002749,0.010426,9.369521,0.082512,16.068179,24.988558,1.819298
std,0.0,3.358059,8.781505,1.995766,1537.450911,479.226780,488.745090,32.518352,7.386172,488.049150,...,494.370414,511.764546,34.263520,0.052359,0.101576,21.059500,3.196129,40.576985,40.401778,15.583226
min,2015.0,1.000000,1.000000,1.000000,1.000000,11.000000,1.000000,-31.000000,1.000000,1.000000,...,1.000000,1.000000,-77.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,3.000000,8.000000,2.000000,556.000000,918.000000,921.000000,-4.000000,10.000000,938.000000,...,1135.000000,1127.000000,-12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,6.000000,16.000000,4.000000,1266.500000,1315.000000,1327.000000,-1.000000,13.000000,1341.000000,...,1535.000000,1532.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,3.000000,11.000000,0.000000
75%,2015.0,9.000000,23.000000,6.000000,2494.000000,1740.000000,1752.000000,8.000000,18.000000,1806.000000,...,1941.000000,1940.000000,8.000000,0.000000,0.000000,12.000000,0.000000,16.000000,33.000000,0.000000
max,2015.0,12.000000,31.000000,7.000000,6894.000000,2359.000000,2400.000000,1670.000000,178.000000,2400.000000,...,2359.000000,2400.000000,1665.000000,1.000000,1.000000,451.000000,440.000000,1665.000000,823.000000,916.000000


### Understanding the data types of the columns

**Question 2**:

* First, classify the *kind* of data each column contains. Create a function `data_kinds` of zero variables which outputs a (hard-coded) dictionary of data kinds, keyed by column name, with values `Q`, `O`, `N` (for 'Quantitative', 'Ordinal', or 'Nominal').

* Second, decide the best data *type* for each column. Create a function `data_types` of zero variables which outputs a (hard-coded) dictionary of data types, keyed by column name, with values `str`, `int`, `float`, `bool`. 

*Remark 1*: A column which *should* be `int`s, but contains `NaN`, *must* be a float column. See Lecture 2 notes an explanation of `NaN` and data-types.

*Remark 2*: As with real data, some data processing decisions may be ambiguous here. Make a best decision using the information available to you. It may be helpful to (re)read the relevant [section of the textbook](https://afraenkel.github.io/practical-data-science/03/kinds-of-data.html). 
* Certain answers *may* have more than one correct answer (in these cases, more than one choice gets full credit),
* All answers will be graded for partial credit (some wrong answers are more wrong than other).
There are many columns, so don't worry about the correctness of any given one; do make sure you are thinking about what's contained in a column critically!

In [34]:
type_obj = (flights.dtypes.values) == np.array([object()]).dtype
flights[np.array(flights.dtypes[type_obj].index)]

,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CANCELLATION_REASON
0,DL,N693DL,SAN,SLC,NaN
1,OO,N930SW,SAN,LAX,NaN
2,WN,N7707C,SAN,BWI,NaN
3,WN,N461WN,SAN,MDW,NaN
4,UA,N69804,SAN,DEN,NaN
5,UA,N24212,SAN,EWR,NaN
6,AA,N496AA,SAN,DFW,NaN
7,NK,N534NK,PDX,SAN,NaN
8,DL,N723TW,SAN,ATL,NaN
9,US,N524UW,SAN,PHX,NaN


In [29]:
flights.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME           int64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

In [40]:
flights['WHEELS_ON'].value_counts()

2016.0    211
1917.0    207
1913.0    206
2030.0    201
1922.0    199
2028.0    198
2027.0    195
1211.0    191
1208.0    190
2033.0    190
1246.0    188
936.0     188
2031.0    186
2029.0    186
1918.0    186
933.0     186
1214.0    184
1903.0    184
2042.0    184
1924.0    183
1939.0    182
1156.0    182
1200.0    182
2100.0    181
1428.0    181
1250.0    180
1426.0    180
930.0     180
1204.0    180
1157.0    180
         ... 
351.0       1
256.0       1
314.0       1
336.0       1
252.0       1
158.0       1
302.0       1
311.0       1
251.0       1
214.0       1
211.0       1
315.0       1
217.0       1
209.0       1
348.0       1
323.0       1
201.0       1
347.0       1
245.0       1
257.0       1
330.0       1
300.0       1
344.0       1
226.0       1
356.0       1
213.0       1
234.0       1
304.0       1
212.0       1
255.0       1
Name: WHEELS_ON, Length: 1388, dtype: int64

### Read in the typed flights data

Read in the flights data using your dictionary of data-types in `read_csv`. This both speeds up parsing, as well as gives you the correct data-types upon reading (which columns would pandas *parse incorrectly* if you didn't use a `dtype` dictionary?)

In [56]:
# Run this cell
dtypes = proj.data_types()
flights = pd.read_csv(to_from_san_filepath, dtype=dtypes)
flights.head(10)
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140414 entries, 0 to 140413
Data columns (total 31 columns):
YEAR                   140414 non-null int64
MONTH                  140414 non-null int64
DAY                    140414 non-null int64
DAY_OF_WEEK            140414 non-null int64
AIRLINE                140414 non-null object
FLIGHT_NUMBER          140414 non-null int64
TAIL_NUMBER            140205 non-null object
ORIGIN_AIRPORT         140414 non-null object
DESTINATION_AIRPORT    140414 non-null object
SCHEDULED_DEPARTURE    140414 non-null int64
DEPARTURE_TIME         139010 non-null float64
DEPARTURE_DELAY        139010 non-null float64
TAXI_OUT               138959 non-null float64
WHEELS_OFF             138959 non-null float64
SCHEDULED_TIME         140414 non-null int64
ELAPSED_TIME           138564 non-null float64
AIR_TIME               138564 non-null float64
DISTANCE               140414 non-null int64
WHEELS_ON              138864 non-null float64
TAXI_IN         

In [64]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140414 entries, 0 to 140413
Data columns (total 31 columns):
YEAR                   140414 non-null int64
MONTH                  140414 non-null int64
DAY                    140414 non-null int64
DAY_OF_WEEK            140414 non-null int64
AIRLINE                140414 non-null object
FLIGHT_NUMBER          140414 non-null int64
TAIL_NUMBER            140205 non-null object
ORIGIN_AIRPORT         140414 non-null object
DESTINATION_AIRPORT    140414 non-null object
SCHEDULED_DEPARTURE    140414 non-null int64
DEPARTURE_TIME         139010 non-null float64
DEPARTURE_DELAY        139010 non-null float64
TAXI_OUT               138959 non-null float64
WHEELS_OFF             138959 non-null float64
SCHEDULED_TIME         140414 non-null int64
ELAPSED_TIME           138564 non-null float64
AIR_TIME               138564 non-null float64
DISTANCE               140414 non-null int64
WHEELS_ON              138864 non-null float64
TAXI_IN         

**Question 3 (Basic Stats):**

Define a function `basic_stats` that takes `flights` and outputs a dataframe that contains statistics for flights arriving/departing for SAN. That is, the output should have two rows, indexed by `ARRIVING` and `DEPARTING`, and have the following columns:

1. number of arriving/departing flights to/from SAN (`count`).
2. mean flight (arrival) delay of arriving/departing flights to/from SAN (`mean_delay`).
3. median flight (arrival) delay of arriving/departing flights to/from SAN (`median_delay`).
4. the airline code of the airline with the longest flight (arrival) delay among all flights arriving/departing to/from SAN (`airline`).
5. a list of the three months with the greatest number of arriving/departing flights to/from SAN, sorted from greatest to least (`top_months`).

*Remark:* Null values should not be considered when computing statistics; however, think about whether e.g. the average flight delay is likely higher or lower that the "true mean" by making this choice.

*Hint*: Use `groupbby` and the fact that `aggregate` can take in a dictionary as an argument.

In [66]:
fp = os.path.join('data', 'to_from_san.csv')
dtypes = proj.data_types()
flights = pd.read_csv(fp, dtype=dtypes)
out = proj.basic_stats(flights)
print(out.index.tolist() == ['ARRIVING', 'DEPARTING'])
cols = ['count', 'mean_delay', 'median_delay', 'airline', 'top_months']
print(out.columns.tolist() == cols)
display(out)
flights.head(10)

True
True


,count,mean_delay,median_delay,airline,top_months
ARRIVING,70207,3.328988,-5.0,AA,"[7, 8, 6]"
DEPARTING,70207,3.676826,-5.0,AA,"[7, 8, 6]"


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,1,4,UA,1568,N24212,SAN,EWR,620,...,1358.0,-42.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,1,1,4,AA,1390,N496AA,SAN,DFW,620,...,1101.0,-24.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,1,4,NK,470,N534NK,PDX,SAN,620,...,828.0,-12.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,1,4,DL,1592,N723TW,SAN,ATL,629,...,1311.0,-19.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,1,1,4,US,567,N524UW,SAN,PHX,630,...,900.0,7.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


### Understanding flight delays: Departures, Arrivals, and everything in-between

**Question 4**

Often `DEPARTURE_DELAY` is thought to be the main cause of a flight delay -- i.e., when the flight is late pushing off from the gate. 

However, there are other ways that flights can be late: waiting on the tarmac, headwinds, turbulence, circling a busy airport, and waiting for a gate after landing. First, we will analyze all the ways in which a flight can be delayed.

* First, create a function `depart_arrive_stats` that takes in a dataframe like `flights` and calculates the following quantities in a series, indexed by `late1`, `late2`, `late3`:
    - The proportion of flights from/to SAN that leave late, but arrive early or on-time (`late1`).
    - The proportion of flights from/to SAN that leaves early, or on-time, but arrives late (`late2`).
    - The proportion of flights from/to SAN that both left late and arrived late (`late3`).
    
* Second, create a function `depart_arrive_stats_by_month` that takes in a dataframe like `flights` and calculates the quantities above broken down by *month*. That is, the output is a dataframe, indexed by `MONTH`, with columns given by `late1`, `late2`, `late3`.

*Remark:* Does this question reveal any data quality issues? Can you pinpoint when these issues occur?

In [68]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140414 entries, 0 to 140413
Data columns (total 31 columns):
YEAR                   140414 non-null int64
MONTH                  140414 non-null int64
DAY                    140414 non-null int64
DAY_OF_WEEK            140414 non-null int64
AIRLINE                140414 non-null object
FLIGHT_NUMBER          140414 non-null int64
TAIL_NUMBER            140205 non-null object
ORIGIN_AIRPORT         140414 non-null object
DESTINATION_AIRPORT    140414 non-null object
SCHEDULED_DEPARTURE    140414 non-null int64
DEPARTURE_TIME         139010 non-null float64
DEPARTURE_DELAY        139010 non-null float64
TAXI_OUT               138959 non-null float64
WHEELS_OFF             138959 non-null float64
SCHEDULED_TIME         140414 non-null int64
ELAPSED_TIME           138564 non-null float64
AIR_TIME               138564 non-null float64
DISTANCE               140414 non-null int64
WHEELS_ON              138864 non-null float64
TAXI_IN         

In [75]:
total_num = flights.shape[0]
late1 = flights[(flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] <= 0)].shape[0] / total_num
late2 = flights[(flights['DEPARTURE_DELAY'] <= 0) & (flights['ARRIVAL_DELAY'] > 0)].shape[0] / total_num
late3 = flights[(flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] > 0)].shape[0] / total_num
pd.Series([late1, late2, late3], index=['late1', 'late2', 'late3'])

late1    0.119853
late2    0.089329
late3    0.278804
dtype: float64

In [78]:
fp = os.path.join('data', 'to_from_san.csv')
dtypes = proj.data_types()
flights = pd.read_csv(fp, dtype=dtypes)
out = proj.depart_arrive_stats(flights)
print(out.index.tolist() == ['late1', 'late2', 'late3'])
print(isinstance(out, pd.Series))
print(out.max() < 0.30)

True
True
True


In [127]:
total = flights[['MONTH', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']].dropna().groupby('MONTH').count()
late1 = (flights.loc[(flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] <= 0), ['MONTH', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']].groupby('MONTH').count() / total)['ARRIVAL_DELAY']
late2 = (flights.loc[(flights['DEPARTURE_DELAY'] <= 0) & (flights['ARRIVAL_DELAY'] > 0), ['MONTH', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']].groupby('MONTH').count() / total)['ARRIVAL_DELAY']
late3 = (flights.loc[(flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] > 0), ['MONTH', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']].groupby('MONTH').count() / total)['ARRIVAL_DELAY']
pd.DataFrame({
    'late1': late1,
    'late2': late2,
    'late3': late3
})

,late1,late2,late3
MONTH,,,
1,0.120199,0.096557,0.273082
2,0.120608,0.098984,0.296113
3,0.132430,0.090545,0.262088
4,0.117508,0.095908,0.266672
5,0.115600,0.084569,0.288809
6,0.123076,0.089660,0.336262
7,0.116699,0.096554,0.339665
8,0.125257,0.087893,0.275596
9,0.111012,0.088369,0.186920


In [128]:
fp = os.path.join('data', 'to_from_san.csv')
dtypes = proj.data_types()
flights = pd.read_csv(fp, dtype=dtypes)
out = proj.depart_arrive_stats_by_month(flights)
print(out.columns.tolist() == ['late1', 'late2', 'late3'])
print(set(out.index) <= set(range(1, 13)))

True
True


### Flight delays and day of the week

**Question 5**

Next, we'd like to understand the flight traffic to/from SAN by day of the week. Day of the week is specified by integers 1 through 7; verify for yourself which integer corresponds to which day (hint: you have the *date* for each flight as well!).

Next create two functions to understand both the amount of traffic and the average flight delay of flights for each airline by day-of-the week. We both want to understand *presence* each airline has as well as their *performance*.

1. Create a function `cnts_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe that answers the following question: Given any `AIRLINE` and `DAY_OF_WEEK`, how many flights were there (in 2015)?


2. Create a function `mean_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe that answers the following question: Given any `AIRLINE` and `DAY_OF_WEEK`, what is the average `ARRIVAL_DELAY` (in 2015)?

Both dataframes should have a column for each distinct value of `AIRLINE` and a row for each `DAY_OF_WEEK`.

*Hint:* Both `groupby` and `pivot` should be useful here!

Your output should have the *form* of the table below (not the entries themselves!)

<img src="pivot.png" />

In [140]:
flights.head()
# flights['DAY'].value_counts()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140414 entries, 0 to 140413
Data columns (total 31 columns):
YEAR                   140414 non-null int64
MONTH                  140414 non-null int64
DAY                    140414 non-null int64
DAY_OF_WEEK            140414 non-null int64
AIRLINE                140414 non-null object
FLIGHT_NUMBER          140414 non-null int64
TAIL_NUMBER            140205 non-null object
ORIGIN_AIRPORT         140414 non-null object
DESTINATION_AIRPORT    140414 non-null object
SCHEDULED_DEPARTURE    140414 non-null int64
DEPARTURE_TIME         139010 non-null float64
DEPARTURE_DELAY        139010 non-null float64
TAXI_OUT               138959 non-null float64
WHEELS_OFF             138959 non-null float64
SCHEDULED_TIME         140414 non-null int64
ELAPSED_TIME           138564 non-null float64
AIR_TIME               138564 non-null float64
DISTANCE               140414 non-null int64
WHEELS_ON              138864 non-null float64
TAXI_IN         

In [141]:
df = flights.pivot_table(
    values  = "DAY",
    index   = "DAY_OF_WEEK",
    columns = "AIRLINE",
    aggfunc = "count",
    fill_value = 0
)
df

AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
1,1915,1564,384,1753,218,96,531,1859,2369,495,536,9238
2,1872,1345,359,1647,210,96,529,1829,2211,463,544,9300
3,1922,1428,359,1719,227,96,530,1814,2293,472,542,9328
4,1888,1551,380,1729,220,96,531,1835,2328,510,531,9108
5,1859,1449,376,1701,208,94,518,1813,2279,505,523,9006
6,1755,1476,270,1433,195,94,518,1837,1743,496,365,7201
7,1886,1465,387,1726,214,96,532,1889,2172,503,443,8587


In [142]:
df = flights.pivot_table(
    values  = "ARRIVAL_DELAY",
    index   = "DAY_OF_WEEK",
    columns = "AIRLINE",
    aggfunc = "mean",
    fill_value = 0
)
df

AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
1,2.245981,-2.652286,0.928571,-3.096904,3.233945,5.842105,9.222656,7.405257,3.081280,1.764344,12.804878,6.355997
2,0.791037,-4.535474,4.507123,-2.988408,4.975728,7.385417,10.669903,5.109574,5.617431,1.354626,4.645522,5.394194
3,1.811808,-4.030899,2.735043,-2.437098,5.634361,16.833333,6.304264,5.566741,3.686344,-2.145299,0.631970,3.556762
4,4.076552,-2.180995,3.152406,-2.339721,4.531818,7.552083,7.931298,7.971901,4.957124,1.198397,8.277040,6.582300
5,1.235679,-1.040887,3.329759,-5.699528,4.830918,8.021505,7.242718,5.920156,1.756997,3.017928,9.325536,7.961504
6,-0.427912,-3.283673,-1.854478,-3.608726,0.119171,7.159574,6.943026,3.746961,-1.338924,-2.227926,0.101370,1.684625
7,1.611924,-2.318213,2.422043,-4.857392,6.112150,5.229167,8.754753,8.570806,-0.190121,2.257545,12.594533,6.867543


In [143]:
fp = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(fp, nrows=100)
out = proj.cnts_by_airline_dow(flights)
print(set(out.columns) == set(flights['AIRLINE'].unique()))
print(set(out.index) == set(flights['DAY_OF_WEEK'].unique()))
print((out >= 0).all().all())

True
True
True


In [145]:
fp = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(fp, nrows=100)
out = proj.mean_by_airline_dow(flights)
print(set(out.columns) == set(flights['AIRLINE'].unique()))
print(set(out.index) == set(flights['DAY_OF_WEEK'].unique()))

True
True


### Understanding null values in the flights data

**Question 6 (Missing by Design)**

Now we would like to understand how data is missing in the flights data. First, compute the proportion of each column of `flights` which are non-null. Do not turn this in, but it will be useful information in doing the next few problems.

Recall that a column is *missing by design* if you can determine when the entry of a column is missing based solely on other data in the same row. That is
* there is *no randomness* in determining when an entry is missing.
* you can describe when the column is missing a value with a logical (not random) condition.
* you can express which rows will have missing values in terms of logical statements about the *other* columns in the same row.

For this question, verify the following columns are *missing by design*:
* The column `ARRIVAL_DELAY` is *missing by design*. Create a function `predict_null_arrival_delay` that doesn't depend on the values of `ARRIVAL_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `ARRIVAL_DELAY` is null; otherwise it returns `False`.
    - Since the function doesn't depend on `ARRIVAL_DELAY`, it should work on a row even if the `ARRIVAL_DELAY` index is dropped.
    - You can check your function by using `flights.drop('ARRIVAL_DELAY', axis=1).apply(predict_null, axis=1)` and compare it to the `ARRIVAL_DELAY` column!


* The column `AIRLINE_DELAY` is *missing by design*. As above, create a function `predict_null_airline_delay` that doesn't depend on the values of `AIRLINE_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `AIRLINE_DELAY` is null; otherwise it returns `False`.


**Question 7 (Missingness Types)**

Now we'd like to determine missingness of the column `DEPARTURE_DELAY`. In particular, we'd like to perform a permutation test to determine the missingness of `DEPARTURE_DELAY` dependent on the column `AIRLINE`.

* Create a function `perm4missing`:
    - that takes in `flights`, a column `col`, and a number `N` and 
    - returns the p-value of the test (using `N` simulations) that determines if `DEPARTURE_DELAY` is MAR dependent on `col`. That is `perm4missing(flights, 'AIRLINE', N)` should return the p-value for the test above.
    - *Remark*: to help your work, create helper functions whose output you can plot, to assess the correctness of your p-value!
    
* Use your function above to determine the columns `col` for which "`DEPARTURE_DELAY` is MAR dependent on `col`" using a significance level of 0.01. Only consider the categorical columns `YEAR`,`DAY_OF_WEEK`, `AIRLINE`,`DIVERTED`, `CANCELLATION_REASON`. Return your answer in a (hard-coded) list returned by a function called `dependent_cols`.

* Create a function `missing_types` of zero variables, which:
    - Returns a Series, indexed by the following columns of `flights`: `CANCELLED`, `CANCELLATION_REASON`, `TAIL_NUMBER`, `ARRIVAL_TIME`.
    - The values should contain the most-likely missingness type of each column. 
    - The values of this Series should be `MD, MCAR, MAR, MNAR, NaN` (use `NaN` if there are no missing values). 


# Simpson's Paradox: JetBlue vs Southwest

The remainder of the questions investigates the presence of Simpson's paradox in the flights dataset. Read through the final slides of lecture 05, as well as [the book](https://afraenkel.github.io/practical-data-science/05/understanding-aggregations.html#simpsons-paradox) for a summary of Simpson's Paradox and related links.

The csv file `southwest_vs_jetblue.csv` contains all Southwest and JetBlue flights in 2015.

In this dataset, we are going to verify the following occurrences of Simpson's Paradox: For a given set of airports,
* The average departure delay of Southwest is greater than (or less than) the average departure delay of JetBlue.
* Airport by airport, the average departure delay of Southwest is *less* than (or greater than) the average departure delay of JetBlue.

That is, the inequalities of the average flight delays between the two airlines are reversed when viewed at the level of each airport. In fact this reversal holds for *every* airport being considered.

In [ ]:
jb_sw_filepath = os.path.join('data', 'jetblue_or_sw.csv')
dtype = proj.data_types()

# The `usecols` keyword:
# choose *only* the columns you need to reduce your memory footprint!
usecols = ...

jb_sw = pd.read_csv(jb_sw_filepath, dtype=dtype, usecols=usecols)

**Question 8**

Filter the dataset `jb_sw` to flights *originating* from the following 10 airports: ABQ, BDL, BUR, DCA, MSY, PBI, PHX, RNO, SJC, SLC.

Illustrate Simpson's paradox with this table:
* Calculate the proportion of each airline's flights that are delayed (at each of the 10 airports):
    - Create a function `prop_delayed_by_airline` that takes in a dataframe like `jb_sw` and returns a DataFrame indexed by airline that contains the proportion of each airline's flights that are delayed.
* Calculate these proportions across all airports in the dataset (at each of the 10 airports):
    - Create a function `prop_delayed_by_airline_airport` that takes in a dataframe like `jb_sw` and returns a DataFrame, with columns given by airports, indexed by airline, that contains the proportion of each airline's flights that are delayed at each airport.

*Remark:* For the purpose of this question, a canceled flight is **not** delayed.

Verify that Simpson's paradox is present in this output! 

Can you explain *why* Simpson's paradox is occurring? (Hint: where are these airports located? Which have the most flights?)

**Question 9**

Your work above illustrates Simpson's paradox on the specific dataset of flights originating from 10 specific airports. However, this still requires you to look at two dataframe to see if the paradox is present. Now, you will create a function that verifies Simpson's paradox in general. You will do this by writing code to compare the two dataframes, instead of inspecting them manually.

Create a function `verify_simpson` that returns a boolean output regarding if the paradox is present.
```
verify_simpson(df, group1, group2, occur)
```
- df is a dataframe (e.g. jb_sw),
- group1 is the first group being aggregated against (e.g. `AIRLINE`),
- group2 is the second group being aggregated against (e.g. `ORIGIN_AIRPORT`),
- occur is a column with values {0, 1}, denoting if an event occurred for that individual.
  (e.g. "1 if flight was delayed" and "0 if flight was not delayed")

`verify_simpson` should return `True` only if there is a reversal for *every* value of `group2` (e.g. for every airport).

Example:

Consider the following dataframe `df` with columns `treatment`, `stone_size`, and `success`:

|treatment|stone_size|success|
|---|---|---|
|A|small|1|
|B|small|1|
|...|...|...|
|A|large|0|
|B|small|0|
|B|small|1|

`df` corresponds to the following diagram:
<img src="https://miro.medium.com/max/996/1*IfVjfdGD7RPwLDC6WzT9Uw.png" style="width: 300px"/>

Here, `verify_simpson(df, 'treatment', 'stone_size', 'success')` should return `True`.

Verify that you function works on the previous question!

### Bonus problem (worth zero points)

This question is for fun and explores a very data-science type problem: can we automate finding examples of Simpson's Paradox? This is an active area of research (see for example: https://arxiv.org/pdf/1801.04385.pdf), but is a very accessible problem. While totally optional, this question can lead to pretty interesting self-driven projects!

**Question 10 (Searching for Simpson's Paradox):**

As you observed from the reading in the lecture notes, Simpson's Paradox often occurs due to some confounding factor among the columns of a dataset. In the case of gender balance in student admissions to academic departments at UC Berkeley, the confounding factor was the admission rate (i.e. how hard it is to gain admission to a department).

What might be a confounding factor be for flight delays among airports in question 8? Now you are going to write code to discover instances of Simpson's Paradox; that is, you will programmatically find interesting relationships present in the data.

Given the dataset `jb_sw`, we'd like to find new groups of airports, as in question 8, for which the statistics of flight delays between JetBlue and Southwest satisfy Simpson's Paradox.

Create a function `search_simpsons` that takes in the `jb_sw` dataset and a number `N`, and returns a list of `N` airports for which the proportion of flight delays between JetBlue and Southwest satisfies Simpson's Paradox.
- Only consider airports that have '3 letter codes',
- Only consider airports that have at least one JetBlue *and* Southwest flight.

*Remark 1:* Iterate through groups of airports of size `N` using `itertools.combinations` until you find a group that works. Make sure your function finishes, even if it doesn't find something.

*Remark 2:* You should be using your work from Question 9!

# Congratulations, you finished the project!

### Before you submit:
* Be sure you run the doctests on all your code in project01.py
* Be sure your free repsonse questions are all answered, readable, and that you haven't changed the cells outside the horizontal lines!

### To submit:
* **Upload the .py file to gradescope**